In [34]:
import pandas as pd
import time

In [35]:
start=time.perf_counter()

In [36]:
from DatabaseCreator import DatabaseCreator
import CJDH_local_settings

#Run Database Creator
if __name__ == "__main__":
    db_creator = DatabaseCreator(db_settings=CJDH_local_settings.local_settings['FPL_Points_Predictor'])
    fpl_engine = db_creator.get_engine_for("fpl_data_analysis")

In [37]:
#add this info to a settings dict?
stats_current_gameweek = 11
picks_current_gameweek = 12
current_season = 20252026

In [38]:
print("Check status of available players")
view = db_creator.run_sql(f"""
    
    WITH predictions_status AS (
    SELECT *
    FROM predictions
    LEFT JOIN player_status ON predictions.element = player_status.id
    )
    SELECT
        web_name,
        status,
        chance_of_playing_this_round,
        event,
        position,
        team_name,
        opp_team_name,
        xpoints
    FROM predictions_status
    WHERE in_my_team = 0
        AND in_any_team = 0
        AND event = {picks_current_gameweek}
        AND status = 'a'
    ORDER BY xpoints DESC
""")
view.head(5)

Check status of available players


,web_name,status,chance_of_playing_this_round,event,position,team_name,opp_team_name,xpoints
0,Pau,a,NaN,12.0,DEF,Aston Villa,Leeds,4.744876
1,O’Reilly,a,100.0,12.0,DEF,Man City,Newcastle,4.593046
2,Zubimendi,a,75.0,12.0,MID,Arsenal,Spurs,4.576382
3,Mukiele,a,NaN,12.0,DEF,Sunderland,Fulham,4.457746
4,McGinn,a,NaN,12.0,MID,Aston Villa,Leeds,4.452857


In [39]:
print("When to use first TRIPLE CAPTAIN chip (gws1-19)")
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        event,
        position,
        team_name,
        opp_team_name,
        xpoints
    FROM predictions
    WHERE in_my_team = 1
      AND event BETWEEN {picks_current_gameweek} AND 19
    ORDER BY xpoints DESC
""")
view.head(5)

When to use first TRIPLE CAPTAIN chip (gws1-19)


,player_name_id,event,position,team_name,opp_team_name,xpoints
0,Gabriel dos Santos Magalhães,16.0,DEF,Arsenal,Wolves,6.464054
1,Gabriel dos Santos Magalhães,12.0,DEF,Arsenal,Spurs,6.026942
2,Ibrahima Konaté,18.0,DEF,Liverpool,Wolves,5.453509
3,Matheus Nunes,13.0,DEF,Man City,Leeds,5.439307
4,Bruno Borges Fernandes,19.0,MID,Man Utd,Wolves,5.413998


In [40]:
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        in_my_team,
        position,
        team_name,
        ROUND(AVG(opp_team_elo),1) AS avg_opp_team_elo,
        ROUND(SUM(xpoints),2) AS total_xpoints,
        ROUND(AVG(mins_pred),1) AS avg_mins_pred,
        ROUND(AVG(xgp90_pred),3) AS avg_xgp90_pred,
        ROUND(AVG(xap90_pred),3) AS avg_xap90_pred,
        ROUND(AVG(xgcp90_pred),3) AS avg_xgcp90_pred
    FROM predictions
    WHERE (in_any_team = 0 OR in_my_team = 1)
      AND event BETWEEN {picks_current_gameweek} AND {picks_current_gameweek+5}
      -- AND position = 'FWD'
    GROUP BY player_name_id, position, team_name, in_my_team
    ORDER BY total_xpoints DESC
""")
view.head(20)

,player_name_id,in_my_team,position,team_name,avg_opp_team_elo,total_xpoints,avg_mins_pred,avg_xgp90_pred,avg_xap90_pred,avg_xgcp90_pred
0,Enzo Fernández,0,MID,Chelsea,1045.7,32.92,77.9,0.359,0.206,1.197
1,Gabriel dos Santos Magalhães,1,DEF,Arsenal,1056.7,31.93,82.2,0.094,0.165,0.983
2,Nico O'Reilly,0,DEF,Man City,1024.1,30.10,69.9,0.060,0.103,0.943
3,Bruno Borges Fernandes,1,MID,Man Utd,1044.4,29.87,84.5,0.252,0.175,1.342
4,Matheus Nunes,1,DEF,Man City,1024.1,29.48,82.5,0.046,0.064,0.932
5,Dominik Szoboszlai,1,MID,Liverpool,1011.4,27.99,82.6,0.178,0.162,1.069
6,Ibrahima Konaté,1,DEF,Liverpool,1011.4,27.78,81.6,0.063,0.056,1.092
7,Sepp van den Berg,0,DEF,Brentford,1016.8,27.55,84.1,0.062,0.046,1.100
8,Yankuba Minteh,1,MID,Brighton,1074.6,27.46,83.0,0.186,0.158,1.352
9,Michael Kayode,0,DEF,Brentford,1016.8,27.45,68.2,0.032,0.112,1.107


In [41]:
print(f"\nBest available players to replace worst in my team over next 5 GWs: {picks_current_gameweek+1} to 38")#{picks_current_gameweek + 5}
view = db_creator.run_sql(f"""
    WITH 
    summed AS (
        SELECT 
            player_name_id,
            team_name,
            position,
            in_my_team,
            in_any_team,
            SUM(xpoints) AS total_xpoints
        FROM predictions
        WHERE event BETWEEN {picks_current_gameweek+1} AND 38 --{picks_current_gameweek + 5}
        GROUP BY player_name_id, team_name, position, in_my_team, in_any_team
    ),

    worst_gk AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'GK'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_gk AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'GK'
    ),

    worst_def AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'DEF'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_def AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'DEF'
    ),

    worst_mid AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'MID'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_mid AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'MID'
    ),

    worst_fwd AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'FWD'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_fwd AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'FWD'
    )

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_gk b
    CROSS JOIN worst_gk w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_def b
    CROSS JOIN worst_def w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_mid b
    CROSS JOIN worst_mid w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_fwd b
    CROSS JOIN worst_fwd w
    ORDER BY xpoints_diff DESC
""")

view.head(20)



Best available players to replace worst in my team over next 5 GWs: 13 to 38


,position,best_team,best_player_id,best_xpoints,worst_player_id,worst_xpoints,xpoints_diff
0,FWD,Burnley,Zian Flemming,101.910481,Chris Wood,45.805684,56.104797
1,FWD,West Ham,Callum Wilson,86.426577,Chris Wood,45.805684,40.620894
2,FWD,Chelsea,Marc Guiu Paz,74.684318,Chris Wood,45.805684,28.878634
3,FWD,Spurs,Richarlison de Andrade,74.532334,Chris Wood,45.805684,28.726650
4,FWD,Fulham,Raúl Jiménez Rodríguez,71.143661,Chris Wood,45.805684,25.337978
5,MID,Chelsea,Enzo Fernández,139.624073,Dango Ouattara,114.969864,24.654209
6,DEF,Man City,Nico O'Reilly,127.620528,James Tarkowski,104.007373,23.613155
7,GK,Liverpool,Giorgi Mamardashvili,99.889623,Nick Pope,78.322246,21.567377
8,FWD,Everton,Norberto Bercique Gomes Betuncal,67.070373,Chris Wood,45.805684,21.264689
9,GK,Aston Villa,Emiliano Martínez Romero,99.094781,Nick Pope,78.322246,20.772536


In [42]:
print(f"\n Zian Flemming xPoints between {picks_current_gameweek} and {picks_current_gameweek+5}:")
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        position,
        team_name,
        opp_team_name,
        value/10 as value,
        opp_team_elo,
        xpoints,
        mins_pred,
        xgp90_pred,
        xap90_pred,
        xgcp90_pred
    FROM predictions
    WHERE event BETWEEN {picks_current_gameweek} AND {picks_current_gameweek+5}
        AND player_name_id = 'Zian Flemming'
        -- AND value <= 48
    ORDER BY event ASC
""")
view.head(20)



 Zian Flemming xPoints between 12 and 17:


,player_name_id,position,team_name,opp_team_name,value,opp_team_elo,xpoints,mins_pred,xgp90_pred,xap90_pred,xgcp90_pred
0,Zian Flemming,FWD,Burnley,Chelsea,5.3,1140.722252,4.024532,67.275387,0.412036,0.040206,2.461656
1,Zian Flemming,FWD,Burnley,Brentford,5.3,1066.648094,3.910136,67.666095,0.373814,0.040206,1.701776
2,Zian Flemming,FWD,Burnley,Crystal Palace,5.3,1100.561911,3.900686,67.225678,0.373385,0.040206,2.339689
3,Zian Flemming,FWD,Burnley,Newcastle,5.3,1085.038872,3.908826,67.665009,0.373385,0.040206,2.257858
4,Zian Flemming,FWD,Burnley,Fulham,5.3,1016.160200,3.907307,68.726060,0.375127,0.040286,1.509715
5,Zian Flemming,FWD,Burnley,Bournemouth,5.3,1068.336869,3.909491,67.630178,0.373814,0.040206,1.709740


In [43]:
# #Gt member player codes from the latest GW
# query = """
#         WITH
#         currentgw AS (
#             SELECT MAX(gw) as max_gw
#             FROM playergw)
#         SELECT *
#         FROM playergw
#         WHERE gw = currentgw
#         ;
#         """
# with fpl_engine.connect() as conn:
#     viewdf = pd.read_sql_query(query, conn)
# print(viewdf.shape)

In [44]:
# #Query gets only player details from the latest GW
# query = """
#         ALTER TABLE player_details
#             RENAME COLUMN total_points TO total_points_gw;

#         WITH playergw AS (
#             SELECT *,
#             ROW_NUMBER() OVER(
#                     PARTITION BY
#                         season,
#                         code,
#                         event,
#                         fixture_id --Only one record per GW
#                     ORDER BY 
#                         now_datetime_utc DESC) as rn --Only select the latest date
#         FROM playergw
#         ),
#         currentgw AS (
#             SELECT MAX(current_gw) as max_gw
#             FROM playergw
#         )
        
#         SELECT *
#         FROM player_details
#         LEFT JOIN playergw
#             ON player_details.element = playergw.id
#             AND player_details.round = playergw.current_gw
#         CROSS JOIN currentgw
#         WHERE player_details.round = currentgw.max_gw
#             AND playergw.rn = 1
#         ;
#         """
# with engine.connect() as conn:
#     playerdf = pd.read_sql_query(query, conn)
# print(playerdf.shape)

# pd.pivot_table(playerdf, index=['id','web_name','team_name','round'],values='total_points_gw',aggfunc=np.sum).sort_values(by='total_points_gw', ascending=False)

In [45]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#10/11/25: Finished in 0.01 minute(s)

Finished in 0.01 minute(s)
